In [108]:
import requests
import pandas as pd
import io
import numpy as np
import time

In [133]:
def get_side(radiant):
    return 'radiant' if radiant else 'dire'


def get_win(radiant, radiant_win):
    if radiant and radiant_win:
        return 1
    elif radiant and not radiant_win:
        return 0
    elif not radiant and radiant_win:
        return 0
    else:
        return 1


def get_kills(my_team, radiant_score, dire_score):
    if my_team == 'radiant':
        return (radiant_score, dire_score)
    else:
        return (dire_score, radiant_score)



def picks(team, picks_bans):
    radiant_picks = []
    dire_picks = []
    if picks_bans:
        for pick_ban in picks_bans:
            if pick_ban['is_pick']:
                if pick_ban['team'] == 0:
                    radiant_picks.append(pick_ban['hero_id'])
                else:
                    dire_picks.append(pick_ban['hero_id'])
        if team == 'radiant':
            return tuple(radiant_picks)
        else:
            return tuple(dire_picks)
    else:
        return 0,0,0,0,0



def get_my_heroes(my_team, picks_bans):
    return picks(my_team, picks_bans)


def get_oppose_team_heroes(my_team, picks_bans):
    if my_team == 'radiant':
        return picks('dire', picks_bans)
    else:
        return picks('radiant', picks_bans)


def get_gold_adv(my_team, radiant_gold_adv):
    try:
        if my_team == 'radiant':
            return radiant_gold_adv[-1]
        else:
            return - radiant_gold_adv[-1]
    except:
        return 0

def get_xp_adv(my_team, radiant_xp_adv):
    try:
        if my_team == 'radiant':
            return radiant_xp_adv[-1]
        else:
            return - radiant_xp_adv[-1]
    except:
        return 0


def get_tower_status(my_team, tower_status_radiant, tower_status_dire):
    if my_team == 'radiant':
        return tower_status_radiant, tower_status_dire
    else:
        return tower_status_dire, tower_status_radiant

In [109]:

matches_of_team = requests.get('https://api.opendota.com/api/teams/15/matches').json() #team psg lgd


In [111]:
len(matches_of_team)

2595

In [117]:
matches_of_team[124:]

[{'match_id': 6389494746,
  'radiant_win': True,
  'radiant': False,
  'duration': 2091,
  'start_time': 1642779984,
  'leagueid': 13647,
  'league_name': 'Intel World Open (Beijing)',
  'cluster': 225,
  'opposing_team_id': 8261500,
  'opposing_team_name': 'Xtreme Gaming',
  'opposing_team_logo': 'https://steamusercontent-a.akamaihd.net/ugc/1833524088514545233/6F322E215508B25533628B8700F797C2D9658650/'},
 {'match_id': 6389305861,
  'radiant_win': True,
  'radiant': False,
  'duration': 2135,
  'start_time': 1642774427,
  'leagueid': 13647,
  'league_name': 'Intel World Open (Beijing)',
  'cluster': 225,
  'opposing_team_id': 8261500,
  'opposing_team_name': 'Xtreme Gaming',
  'opposing_team_logo': 'https://steamusercontent-a.akamaihd.net/ugc/1833524088514545233/6F322E215508B25533628B8700F797C2D9658650/'},
 {'match_id': 6389079885,
  'radiant_win': True,
  'radiant': True,
  'duration': 3501,
  'start_time': 1642767105,
  'leagueid': 13647,
  'league_name': 'Intel World Open (Beijing)'

In [112]:
matches_list = []

In [134]:
counter = 0
#35
for match in matches_of_team[1200:]:
    if counter == 1000:
        break
    counter += 1

    _match_id = match['match_id']
    _side = get_side(match['radiant'])
    _win = get_win(match['radiant'], match['radiant_win'])
    _opposing_team_id = match['opposing_team_id']

    time.sleep(0.5)
    req = requests.get(f'https://api.opendota.com/api/matches/{_match_id}')

    math_details = req.json()
    _my_team_kills, _opposing_team_kills = get_kills(_side, math_details['radiant_score'], math_details['dire_score'])
    _duration = math_details['duration']
    _my_hero_1, _my_hero_2, _my_hero_3, _my_hero_4, _my_hero_5 = get_my_heroes(_side, math_details['picks_bans'])
    _opposing_hero_1, _opposing_hero_2, _opposing_hero_3, _opposing_hero_4, _opposing_hero_5 = get_oppose_team_heroes(_side, math_details['picks_bans'])
    _gold_adv = get_gold_adv(_side, math_details['radiant_gold_adv'])
    _xp_adv = get_xp_adv(_side, math_details['radiant_xp_adv'])
    _my_tower_status, _opposing_team_tower_status = get_tower_status(_side, math_details['tower_status_radiant'], math_details['tower_status_dire'])

    matches_list.append({
        'match_id': _match_id,
        'opposing_team_id': _opposing_team_id,
        'side': _side,
        'duration': _duration,
        'my_hero_1': _my_hero_1,
        'my_hero_2': _my_hero_2,
        'my_hero_3': _my_hero_3,
        'my_hero_4': _my_hero_4,
        'my_hero_5': _my_hero_5,
        'opposing_hero_1': _opposing_hero_1,
        'opposing_hero_2': _opposing_hero_2,
        'opposing_hero_3': _opposing_hero_3,
        'opposing_hero_4': _opposing_hero_4,
        'opposing_hero_5': _opposing_hero_5,
        'my_team_kills' : _my_team_kills,
        'opposing_team_kills': _opposing_team_kills,
        'gold_adv': _gold_adv,
        'xp_adv': _xp_adv,
        'my_tower_status': _my_tower_status,
        'opposing_team_tower_status': _opposing_team_tower_status,
        'win': _win,
    })



KeyboardInterrupt: 

In [135]:
dataset = pd.DataFrame(matches_list)
dataset

,match_id,opposing_team_id,side,duration,my_hero_1,my_hero_2,my_hero_3,my_hero_4,my_hero_5,opposing_hero_1,...,opposing_hero_3,opposing_hero_4,opposing_hero_5,my_team_kills,opposing_team_kills,gold_adv,xp_adv,my_tower_status,opposing_team_tower_status,win
0,6809621786,7732977,dire,1901,136,11,61,87,17,85,...,43,46,19,28,13,19623,18197,1974,1536,1
1,6809585544,7732977,radiant,1613,137,52,91,18,7,129,...,74,68,10,22,6,24806,29717,1983,1572,1
2,6809420587,6209804,radiant,1672,137,66,114,38,119,11,...,128,120,64,32,9,15500,16130,1983,1536,1
3,6808199906,2163,radiant,2583,33,19,110,13,10,85,...,86,35,2,20,25,10673,18586,1956,288,1
4,6808106195,2163,radiant,2785,33,19,25,87,54,137,...,52,38,99,32,29,14209,32044,1828,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1943,949542385,1924713,radiant,1271,101,78,54,31,11,29,...,67,68,74,0,0,0,0,2046,1796,1
1944,949447904,1924713,dire,2320,78,53,16,68,94,15,...,9,96,33,0,0,0,0,2038,4,1
1945,944216858,1367764,dire,2827,78,53,86,16,45,109,...,100,13,91,0,0,0,0,1828,1540,1
1946,944147749,1367764,radiant,1556,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2039,1540,1


In [136]:
dataset.describe()

,match_id,opposing_team_id,duration,my_hero_1,my_hero_2,my_hero_3,my_hero_4,my_hero_5,opposing_hero_1,opposing_hero_2,opposing_hero_3,opposing_hero_4,opposing_hero_5,my_team_kills,opposing_team_kills,gold_adv,xp_adv,my_tower_status,opposing_team_tower_status,win
count,1.948000e+03,1.948000e+03,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.00000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000
mean,3.873641e+09,2.506611e+06,2240.287988,65.197125,60.783368,60.076489,56.319815,54.438398,63.446099,60.281828,60.953799,56.122690,55.26540,20.333676,17.339322,4628.086242,5433.997433,1652.329055,1503.855749,0.645277
std,1.762814e+09,2.630412e+06,661.714610,36.729192,36.172861,36.636498,36.122457,34.485295,36.855869,36.669727,36.407201,34.567975,34.27738,13.898508,13.495814,15681.334637,19688.552275,577.466681,624.335532,0.478552
min,9.384425e+08,3.000000e+00,715.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,-40596.000000,-61070.000000,0.000000,0.000000,0.000000
25%,2.387757e+09,1.114740e+05,1786.000000,30.000000,28.000000,26.000000,22.000000,22.000000,28.000000,27.000000,27.000000,25.000000,25.00000,8.000000,6.000000,-7600.750000,-8312.750000,1542.000000,1536.000000,0.000000
50%,3.820086e+09,1.554950e+06,2138.000000,68.000000,63.500000,62.000000,53.000000,51.000000,66.000000,60.000000,64.000000,54.000000,52.00000,23.000000,16.000000,6204.500000,5935.000000,1846.000000,1796.000000,1.000000
75%,5.497550e+09,3.331948e+06,2603.000000,97.000000,90.000000,91.000000,88.000000,81.000000,96.000000,90.000000,91.000000,86.000000,82.00000,31.000000,28.000000,17085.500000,20351.500000,1975.000000,1924.000000,1.000000
max,6.809622e+09,8.740926e+06,6544.000000,137.000000,136.000000,136.000000,136.000000,135.000000,137.000000,136.000000,136.000000,135.000000,135.00000,63.000000,61.000000,45685.000000,64710.000000,2047.000000,2047.000000,1.000000


In [147]:
(dataset['opposing_team_tower_status'] == 0).sum()

101